In [28]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
import hvplot.pandas
%matplotlib inline

In [29]:
load_dotenv()

True

In [30]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [31]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2017-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["PYPL","SQ","MELI","FISV"]
fintech_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
fintech_ticker.dropna(inplace=True)
fintech_ticker.head(5)
#apple_ticker.dropna(inplace=True)

FISV                                      MELI  \
                             open    high     low  close     volume    open   
time                                                                          
2017-09-05 00:00:00-04:00  61.290  61.355  60.660  60.93  1071708.0  252.60   
2017-09-06 00:00:00-04:00  61.185  61.330  60.585  61.19  1610582.0  257.67   
2017-09-07 00:00:00-04:00  61.465  61.785  61.085  61.41   864020.0  258.92   
2017-09-08 00:00:00-04:00  61.335  61.730  61.015  61.33   851518.0  258.21   
2017-09-11 00:00:00-04:00  61.700  62.235  61.400  62.15   734684.0  265.08   

                                                              PYPL         \
                             high     low   close    volume   open   high   
time                                                                        
2017-09-05 00:00:00-04:00  257.76  251.51  256.63  453957.0  61.26  61.75   
2017-09-06 00:00:00-04:00  259.35  251.92  258.10  365307.0  61.74  61.99   
2017-09-07 00:00:00-04:00  260.95  256.16  258.68  328688.0  61.33  62.13   
2017-09-08 00:00:00-04:00  262.25  258.21  260.08  255764.0  61.94  62.00   
2017-09-11 00:00:00-04:00  286.66  265.07  285.22  973710.0  61.71  63.26   

                                                       SQ                \
                             low  close     volume   open   high    low   
time                                                                      
2017-09-05 00:00:00-04:00  60.58  61.27  4462105.0  25.75  25.82  25.24   
2017-09-06 00:00:00-04:00  60.90  61.25  4246915.0  25.90  26.17  25.72   
2017-09-07 00:00:00-04:00  61.28  62.00  4141248.0  26.12  26.69  25.86   
2017-09-08 00:00:00-04:00  61.10  61.13  3893906.0  26.50  26.99  26.40   
2017-09-11 00:00:00-04:00  61.62  62.64  5970316.0  26.80  27.66  26.78   

                                             
                           close     volume  
time                                         
2017-09-05 00:00:00-04:00  25.77  4478368.0  
2017-09-06 00:00:00-04:00  26.07  4507494.0  
2017-09-07 00:00:00-04:00  26.31  6699110.0  
2017-09-08 00:00:00-04:00  26.53  5567927.0  
2017-09-11 00:00:00-04:00  27.58  7416518.0

In [32]:
fintech_df = pd.DataFrame()
fintech_df["PayPal"] = fintech_ticker["PYPL"]["close"]
fintech_df["Square"] = fintech_ticker["SQ"]["close"]
#fintech_df["Affirm"] = fintech_ticker["AFRM"]["close"]
fintech_df["Mercadolibre"] = fintech_ticker["MELI"]["close"]
#fintech_df["Johnson&Johnson"] = nasdaq_ticker["JNJ"]["close"]
#apple_df = apple_df.pct_change().dropna()
fintech_df.index = fintech_df.index.date 
fintech_df.dropna(inplace=True)
fintech_df.head()

,PayPal,Square,Mercadolibre
2017-09-05,61.27,25.77,256.63
2017-09-06,61.25,26.07,258.10
2017-09-07,62.00,26.31,258.68
2017-09-08,61.13,26.53,260.08
2017-09-11,62.64,27.58,285.22


In [33]:
csv_path = Path("./stock_data/vixe.csv/")
data = pd.read_csv(csv_path,index_col="Date", parse_dates=True, infer_datetime_format=True)
data.sort_index(ascending = True, inplace = True)
data.dropna(inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-05,11.75,14.06,11.41,12.23,12.23,0.0
2017-09-06,12.27,12.59,11.35,11.63,11.63,0.0
2017-09-07,11.93,12.07,11.32,11.55,11.55,0.0
2017-09-08,11.87,12.60,11.84,12.12,12.12,0.0
2017-09-11,11.38,11.39,10.51,10.73,10.73,0.0


In [34]:
vix_df = pd.DataFrame()
vix_df["Vix"] = data["Close"]
#vix_df.index = fintech_df.index.date 
vix_df.dropna(inplace=True)
vix_df.tail()

,Vix
Date,
2021-08-17,17.910000
2021-08-18,21.570000
2021-08-19,21.670000
2021-08-20,18.559999
2021-08-23,17.150000


In [35]:
data = fintech_df.join(vix_df, how = "inner")
data.sort_index(inplace=True)
data.head()

,PayPal,Square,Mercadolibre,Vix
2017-09-05,61.27,25.77,256.63,12.23
2017-09-06,61.25,26.07,258.10,11.63
2017-09-07,62.00,26.31,258.68,11.55
2017-09-08,61.13,26.53,260.08,12.12
2017-09-11,62.64,27.58,285.22,10.73


In [36]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [37]:
window_size = 100
feature_column = 2
target_column = 2
X, y = window_data(fintech_df, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:3]} \n")
print (f"y sample values:\n{y[:3]}")

X sample values:
[[256.63  258.1   258.68  260.08  285.22  290.79  292.22  287.94  282.56
  289.83  289.96  290.17  281.84  279.46  270.2   269.27  267.18  261.99
  258.93  263.56  264.36  263.8   270.29  268.7   269.87  273.65  274.49
  246.25  244.86  233.02  234.79  242.41  236.    235.72  226.56  225.45
  221.57  228.99  235.96  234.76  240.38  239.02  236.41  267.33  268.795
  265.63  260.09  266.46  271.    267.02  262.58  263.56  262.44  269.58
  265.79  268.759 265.85  271.18  272.61  269.84  267.93  275.32  272.97
  271.76  270.81  272.26  272.68  276.49  288.81  295.06  318.99  321.08
  318.99  326.05  329.27  314.94  318.49  314.58  318.49  318.985 315.61
  314.7   322.58  329.24  325.88  330.64  340.5   337.38  339.59  342.01
  342.47  339.82  340.45  341.88  345.32  352.27  355.31  355.6   359.93
  381.31 ]
 [258.1   258.68  260.08  285.22  290.79  292.22  287.94  282.56  289.83
  289.96  290.17  281.84  279.46  270.2   269.27  267.18  261.99  258.93
  263.56  264.36  263.

In [38]:
print(X)

[[ 256.63  258.1   258.68 ...  355.6   359.93  381.31]
 [ 258.1   258.68  260.08 ...  359.93  381.31  390.37]
 [ 258.68  260.08  285.22 ...  381.31  390.37  386.5 ]
 ...
 [1433.14 1472.13 1509.68 ... 1767.12 1787.66 1757.9 ]
 [1472.13 1509.68 1501.76 ... 1787.66 1757.9  1786.3 ]
 [1509.68 1501.76 1547.78 ... 1757.9  1786.3  1810.23]]


In [39]:
split = int(0.6 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [40]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [41]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
#print (f"X_train sample values:\n{X_train[:5]} \n")
#print (f"X_test sample values:\n{X_test[:5]}")

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [43]:
model = Sequential()

#Adding number_units to the architecture enables the model to far more accurate that with a single digit number
number_units = 100
dropout_fraction = 0.2


# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [44]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 100)          40800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [46]:
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=100, verbose=1)

Epoch 1/20
6/6 [==============================] - 5s 262ms/step - loss: 0.0031
Epoch 2/20
6/6 [==============================] - 2s 289ms/step - loss: 0.0084
Epoch 3/20
6/6 [==============================] - 2s 296ms/step - loss: 0.0014
Epoch 4/20
6/6 [==============================] - 2s 239ms/step - loss: 0.0043
Epoch 5/20
6/6 [==============================] - 2s 256ms/step - loss: 0.0016
Epoch 6/20
6/6 [==============================] - 2s 285ms/step - loss: 0.0011
Epoch 7/20
6/6 [==============================] - 2s 271ms/step - loss: 0.0022
Epoch 8/20
6/6 [==============================] - 2s 265ms/step - loss: 6.5713e-04
Epoch 9/20
6/6 [==============================] - 2s 264ms/step - loss: 9.7328e-04
Epoch 10/20
6/6 [==============================] - 2s 271ms/step - loss: 6.3675e-04
Epoch 11/20
6/6 [==============================] - 2s 271ms/step - loss: 5.6253e-04
Epoch 12/20
6/6 [==============================] - 2s 268ms/step - loss: 7.0741e-04
Epoch 13/20
6/6 [============

In [47]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 1s 30ms/step - loss: 0.0097


0.009653737768530846

In [48]:
predicted = model.predict(X_test)

In [49]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [50]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = fintech_df.index[-len(real_prices): ])
stocks.tail(10)

,Real,Predicted
2021-08-11,1887.57,1458.003906
2021-08-12,1842.11,1475.964233
2021-08-13,1853.47,1495.678955
2021-08-16,1795.00,1515.818359
2021-08-17,1767.12,1534.317749
2021-08-18,1787.66,1549.679321
2021-08-19,1757.90,1561.740234
2021-08-20,1786.30,1570.189209
2021-08-23,1810.23,1575.743896
2021-08-24,1857.01,1579.430420


In [51]:
stocks.hvplot.line(label = "LSTM Model for MercadoLibre", width = 1000, xlabel="Date", ylabel="Price")
#(label = "Predicting Apple's Close with the S&P500 Close - (6Day Window)", xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)